## Tables and Data Source setup

In [1]:
using ODBC
using DataFrames
using DSWB
using Formatting
using URIParser
using JSON

dsn = "dswb-natgeo" # Redshift esetTable(tableRt, tableType = "RESOURCE_TABLE")ndpoint
table = "beacons_4744" # beacon table name
tableRt = "beacons_4744_rt"

# Connect to Beacon Data
setRedshiftEndpoint(dsn)
setTable(table)
setTable(tableRt, tableType = "RESOURCE_TABLE")
;

Redshift endpoint set: 2017-08-29T19:36:25


In [2]:
# Packages

include("/data/notebook_home/Production/Lib/Include-Package-v1.0.jl")
include("/data/notebook_home/Production/Bodies/Individual-Streamline-Body-v2.0.jl")

timeVariables(2017,8,27,0,0,2017,8,27,23,59)
#weeklyTimeVariables(days=7)
#yesterdayTimeVariables()
;

August 27, 2017 00:00:00 to August 27, 2017 23:59:00 Local Time
August 27, 2017 04:00:00 to August 28, 2017 03:59:00 UTC Time


In [3]:
# Packages

include("/data/notebook_home/Production/Bodies/Individual-Streamline-v1.2.jl")
#Include JSON below - include("/data/notebook_home/Production/Bodies/Individual-Streamline-Json-v1.2.jl")

UP = UrlParamsInit()
UP.beaconTable = table
UP.resourceTable = tableRt
UP.pageGroup = "%"
UP.urlRegEx = "%"
UP.urlFull = "%"
#UP.limitRows = 3
UP.limitRows = 250
UP.samplesMin = 10
UP.sizeMin = 10000
UP.usePageLoad=false
UP.timeLowerMs = 0.0 #2000.0
UP.timeUpperMs = 30000.0 #300000.0

SP = ShowParamsInit()
SP.desktop=false
SP.mobile=true
SP.devView=false
SP.criticalPathOnly=true
SP.debug=true
SP.debugLevel = 10   # Tests use even numbers with > tests, make this an odd number or zero

LV = LocalVars(true) #useJSon = true
;

In [4]:

theList = "
{
    \"data\": [

        {\"attributes\": {\"publication_datetime\": \"2017-08-24T04:01:00.000000Z\", \"uri\": \"http://www.nationalgeographic.com/magazine/2017/09/explore-health-food-nutrition-science/\"}, \"id\": \"NGM:54b2dd5d-87fe-4d32-b83f-13924a024928\", \"links\": {\"self\": \"https://relay.nationalgeographic.com/proxy/distribution/api/v1/item/NGM:54b2dd5d-87fe-4d32-b83f-13924a024928/?format=jsonapi\"}, \"type\": \"article:story\"},
        {\"attributes\": {\"publication_datetime\": \"2017-08-24T16:34:26.000000Z\", \"uri\": \"http://news.nationalgeographic.com/2017/08/purple-frog-new-species-discovery-india-monsoon/\"}, \"id\": \"News:7705b1de-01b7-4df2-b6bb-2a5fc20bb157\", \"links\": {\"self\": \"https://relay.nationalgeographic.com/proxy/distribution/api/v1/item/News:7705b1de-01b7-4df2-b6bb-2a5fc20bb157/?format=jsonapi\"}, \"type\": \"article:story\"},
        {\"attributes\": {\"publication_datetime\": \"2017-08-25T00:30:49.000000Z\", \"uri\": \"http://news.nationalgeographic.com/2017/08/ancient-babylonian-trigonometry-tablet-plimpton-322-video-spd/\"}, \"id\": \"News:2cb97edd-061b-4bfe-bd73-b7ef1d9c7908\", \"links\": {\"self\": \"https://relay.nationalgeographic.com/proxy/distribution/api/v1/item/News:2cb97edd-061b-4bfe-bd73-b7ef1d9c7908/?format=jsonapi\"}, \"type\": \"article:story\"}

]
,\"meta\": {
        \"pages\": 2,
        \"feeds\": 44
    },
    \"links\": {\"first\": \"https://relay.nationalgeographic.com/proxy/distribution/feed/v1/jsonapi/?fields=uri%2Cpublication_datetime\u0026publication_datetime=7d\u0026content_type=article%3Astory\u0026page=1\", \"last\": \"https://relay.nationalgeographic.com/proxy/distribution/feed/v1/jsonapi/?fields=uri%2Cpublication_datetime\u0026publication_datetime=7d\u0026content_type=article%3Astory\u0026page=2\", \"next\": \"https://relay.nationalgeographic.com/proxy/distribution/feed/v1/jsonapi/?fields=uri%2Cpublication_datetime\u0026publication_datetime=7d\u0026content_type=article%3Astory\u0026page=2\", \"self\": \"https://relay.nationalgeographic.com/proxy/distribution/feed/v1/jsonapi/?fields=uri%2Cpublication_datetime\u0026publication_datetime=7d\u0026content_type=article%3Astory\"},
    \"jsonapi\": {
        \"version\": \"1.0\"
    }
}
";


In [5]:
if (LV.useJson)
    urlListDF = newPagesList()
    if (SP.debugLevel > 4)
        beautifyDF(urlListDF[1:min(3,end),:])
    end

    topUrlList = urlListDF[:urlgroup]
    topUrls = cleanupTopUrlTable(topUrlList)
    if (SP.debugLevel > 0)
        display(topUrls)
    end
end
;

urlgroup
http://www.nationalgeographic.com/magazine/2017/09/explore-health-food-nutrition-science/
http://news.nationalgeographic.com/2017/08/purple-frog-new-species-discovery-india-monsoon/
http://news.nationalgeographic.com/2017/08/ancient-babylonian-trigonometry-tablet-plimpton-322-video-spd/


3-element DataArrays.DataArray{ASCIIString,1}:
 "/www.nationalgeographic.com/magazine/2017/09/explore-health-food-nutrition-science/"                
 "/news.nationalgeographic.com/2017/08/purple-frog-new-species-discovery-india-monsoon/"              
 "/news.nationalgeographic.com/2017/08/ancient-babylonian-trigonometry-tablet-plimpton-322-video-spd/"

In [6]:
# Time
if (LV.useJson)
    SP.mobile = true
    SP.desktop = false
    UP.orderBy = "time"
    finalUrlTableOutput(TV,UP,SP,topUrls)
end

params u,session id,session start,timestamp,url,initiator type,start time,redirect start,redirect end,fetch start,dns start,dns end,tcp connection start,secure connection start,tcp connection end,request start,response first byte,response last byte,worker start,encoded size,transferred size,decoded size,height,width,x,y,beacon id,compression types,custom metrics 0,custom metrics 1,custom metrics 2,custom metrics 3,custom metrics 4,custom metrics 5,custom metrics 6,custom metrics 7,custom metrics 8,custom metrics 9,domain,geo cc,geo city,geo lat,geo lon,geo netspeed,geo org,geo postalcode,geo rg,http method,http referrer,http version,key,mobile connection type,page group,params bat lvl,params cmet mpulseid,params cpu cnc,params custom0 st,params custom1 st,params custom2 st,params custom3 st,params custom4 st,params custom5 st,params custom6 st,params custom7 st,params custom8 st,params custom9 st,params dom doms,params dom img,params dom ln,params dom res,params dom res slowest,params dom script,params dom sz,params errors,params h t,params if,params mem total,params mem used,params mob bw,params mob mt,params nt con end,params nt con st,params nt dns end,params nt dns st,params nt domcomp,params nt domcontloaded e,params nt domcontloaded st,params nt domint,params nt domloading,params nt fet st,params nt first paint,params nt load end,params nt load st,params nt nav st,params nt nav type,params nt red cnt,params nt red end,params nt red st,params nt req st,params nt res end,params nt res st,params nt spdy,params nt ssl st,params nt unload end,params nt unload st,params nu,params pgu,params r,params r2,params rt abld,params rt blstart,params rt bmr conen,params rt bmr const,params rt bmr domen,params rt bmr domst,params rt bmr fetst,params rt bmr reqst,params rt bmr resen,params rt bmr resst,params rt bmr secst,params rt bstart,params rt cnf conen,params rt cnf const,params rt cnf domen,params rt cnf domst,params rt cnf fetst,params rt cnf reqst,params rt cnf resen,params rt cnf resst,params rt cnf secst,params rt cstart,params rt end,params rt ntvu,params rt obo,params rt quit,params rt sh,params rt si,params rt sl,params rt srst,params rt start,params rt tstart,params rt tt,params scr bpp,params scr dpx,params scr mtp,params scr orn,params scr xy,params u 1,params v,params vis lh,params vis lv,params vis st,proxy address,remote ip,session id 1,session isunload,session latest,session obopages,session pages,session start 1,session totalloadtime,spdy,ssl,timers before dns,timers boomerang,timers boomr fb,timers boomr lat,timers boomr ld,timers boomr to end,timers custom0,timers custom1,timers custom2,timers custom3,timers custom4,timers custom5,timers custom6,timers custom7,timers custom8,timers custom9,timers dns,timers domload,timers domready,timers fb to boomr,timers loaded,timers missing,timers navst to boomr,timers renderstart,timers ssl,timers t configfb,timers t configjs,timers t domloaded,timers t done,timers t load,timers t page,timers t postrender,timers t prerender,timers t resp,timers tcp,timestamp 1,url 1,user agent family,user agent major,user agent minor,user agent mobile,user agent model,user agent os,user agent osversion,user agent raw,warnings,beacon type,site version,ab test,ipv6,user agent manufacturer,user agent device type,user agent isp,geo isp,bandwidth kbps,bandwidth error pc,bandwidth block,headers cf visitor,headers cf ray,headers cf connecting ip,headers x forwarded for,headers x forwarded proto,headers connection,headers host,headers accept encoding,headers accept language,headers accept,headers content length,headers various,params dom img ext,params dom script ext,params ua plt,params ua vnd,http errno,params http method,params http hdr,params http initiator,params h cr,params xhr sync,params rt subres,params bw time,params lat,params lat err,params mob ct,cdim,params nt cinf,params rt ss,errors,tp ga clientid,tp ga utm source,tp ga utm medium,tp ga utm term,tp ga utm content,tp ga utm campaign,tp

pg=%

urlgroup,request count,beacon time,encoded size,errors,session id,timestamp
http://www.nationalgeographic.com/magazine/2017/09/explore-health-food-nutrition-science,60,"2,789","741,324",,89fc4162-0f69-40ba-b2ca-b327c35a1c41-ovcoy9,2017-08-27T15:46:40.305
http://www.nationalgeographic.com/magazine/2017/09/explore-health-food-nutrition-science,114,"17,745","1,169,398",,4016c259-8965-4021-8f83-1dfdefe23260-ovcrz6,2017-08-27T16:48:21.638
http://www.nationalgeographic.com/magazine/2017/09/explore-health-food-nutrition-science,68,"16,392",,,48795985-0995-4afa-b53a-9ce9f3242dc7-ovcs2f,2017-08-27T16:49:59.695
http://www.nationalgeographic.com/magazine/2017/09/explore-health-food-nutrition-science,77,"11,999","932,364",,f606ffe6-a9c0-4301-861a-f900cb5b3a3a-ovcri5,2017-08-27T16:37:30.827
http://www.nationalgeographic.com/magazine/2017/09/explore-health-food-nutrition-science,119,"23,382","1,088,468",,21d15072-62cc-47e5-b61b-e6925a41ce97-ovcriy,2017-08-27T16:37:07.884
http://www.nationalgeographic.com/magazine/2017/09/explore-health-food-nutrition-science,77,"12,607","906,905",,70908de9-bc08-4312-a0f5-e74bfe692f69-ovcrtj,2017-08-27T16:58:08.160
http://www.nationalgeographic.com/magazine/2017/09/explore-health-food-nutrition-science,123,"18,989","942,667",,ad2e172b-aa92-4343-8c0e-f8605350be77-ovcsci,2017-08-27T16:57:13.721
http://www.nationalgeographic.com/magazine/2017/09/explore-health-food-nutrition-science,112,"11,547","886,580",,136c81bc-aa1b-46fe-996b-e10002bf538a-ovcrg4,2017-08-27T16:36:16.189
http://www.nationalgeographic.com/magazine/2017/09/explore-health-food-nutrition-science,102,"23,454","835,794",,aff842fe-7ff2-41c0-89db-b49d7fdb9a73-ovcrlm,2017-08-27T16:39:48.760
http://www.nationalgeographic.com/magazine/2017/09/explore-health-food-nutrition-science,63,"19,122",,,90f84e6a-ecdd-4356-937d-c91981e04d00-ovcrff,2017-08-27T16:36:43.113


 url=%/www.nationalgeographic.com/magazine/2017/09/explore-health-food-nutrition-science/% dev=mobile dr lower=0.0 dr upper=30000.0
part 1 done with 203 records
bt=

url,beacon time,request count,encoded size,samples
/www.nationalgeographic.com/magazine/2017/09/explore-health-food-nutrition-science/,"14,622",80,"906,613",203


...

urlgroup,request count,beacon time,encoded size,errors,session id,timestamp
http://news.nationalgeographic.com/2017/08/ancient-babylonian-trigonometry-tablet-plimpton-322-video-spd,104,"23,520","983,700",,76fe0019-0d09-44c6-8e84-f33a7d6eb808-ovbubh,2017-08-27T04:40:54.689
http://news.nationalgeographic.com/2017/08/ancient-babylonian-trigonometry-tablet-plimpton-322-video-spd,81,"15,394","739,160",,7e757506-7fe9-4507-83ab-6fbd47962508-ovbubj,2017-08-27T04:41:53.217
http://news.nationalgeographic.com/2017/08/ancient-babylonian-trigonometry-tablet-plimpton-322-video-spd,206,"18,903","1,658,233",,23cfcbd6-3338-4ff7-9d5f-5f936b900d0b-ovbv0l,2017-08-27T04:55:52.702
http://news.nationalgeographic.com/2017/08/ancient-babylonian-trigonometry-tablet-plimpton-322-video-spd,100,"9,245","1,022,734",,16347991-9ef8-426c-ab0c-ff63a026f31b-ovbtza,2017-08-27T04:33:23.783
http://news.nationalgeographic.com/2017/08/ancient-babylonian-trigonometry-tablet-plimpton-322-video-spd,91,"14,040","893,705",,c3554e45-c952-4ce1-af90-5110749e608f-ovbqjp,2017-08-27T04:21:36.991
http://news.nationalgeographic.com/2017/08/ancient-babylonian-trigonometry-tablet-plimpton-322-video-spd,150,"13,647","998,929",,d599c70e-47de-40ed-8802-94fa43aabcf5-ovbu6j,2017-08-27T04:37:46.447
http://news.nationalgeographic.com/2017/08/ancient-babylonian-trigonometry-tablet-plimpton-322-video-spd,122,"8,642","1,024,904",,12a44811-9979-4b33-896e-80fd324f0033-ovbtua,2017-08-27T04:30:20.438
http://news.nationalgeographic.com/2017/08/ancient-babylonian-trigonometry-tablet-plimpton-322-video-spd,192,"13,986",,,25ba8cbf-56d7-4af2-a92e-2dba46c84cf7-ovbtkl,2017-08-27T04:23:17.276
http://news.nationalgeographic.com/2017/08/ancient-babylonian-trigonometry-tablet-plimpton-322-video-spd,112,"14,018","973,993",,0719b70d-45da-4ae2-9eb6-46974da87555-ovbto5,2017-08-27T04:26:46.116
http://news.nationalgeographic.com/2017/08/ancient-babylonian-trigonometry-tablet-plimpton-322-video-spd,183,"10,134",,,0adae04d-ce2b-48eb-bf7e-191a656a9a4d-ovbu1o,2017-08-27T04:34:46.771


pg=% url=%/news.nationalgeographic.com/2017/08/ancient-babylonian-trigonometry-tablet-plimpton-322-video-spd/% dev=mobile dr lower=0.0 dr upper=30000.0
part 1 done with 

url,beacon time,request count,encoded size,samples
/news.nationalgeographic.com/2017/08/ancient-babylonian-trigonometry-tablet-plimpton-322-video-spd/,"15,360",90,"1,029,832",585


Recent Urls (Sorted By Time Descending; Min Samples 10 ; Top 250 Page Views),Time,Request Made,Page Size,Samples
/news.nationalgeographic.com/2017/08/purple-frog-new-species-discovery-india-monsoon/,"15,704",88,"1,029,102","2,588"
/news.nationalgeographic.com/2017/08/ancient-babylonian-trigonometry-tablet-plimpton-322-video-spd/,"15,360",90,"1,029,832",585
/www.nationalgeographic.com/magazine/2017/09/explore-health-food-nutrition-science/,"14,622",80,"906,613",203


585 records
bt=15360.0 goal=3000.0
